# Libraries

In [16]:
import os
import pandas as pd

from datetime import datetime
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# Functions

### Data

In [17]:
PATH = 'Data/Real/'

In [18]:
def load_data(ind = 0, # which of the file
              type = 'Financial/',  # type = ['Text', 'Financial']
              ):
    
    assert ind < len(os.listdir(PATH + type)) , f'Chose {ind + 1}. file out of *{len(os.listdir(PATH + type))}* files'
    print(f'Loading {os.listdir(PATH + type)[ind]}')
    return pd.read_csv(PATH + type + os.listdir(PATH + type)[ind])

In [ ]:
# Load data and get tickers
X_financial = load_data(1, type = 'Financial/')

TICKERS = X_financial['ticker'].unique()
TICKERS

### Financial feature engineering

In [ ]:
X_financial['timestamp'] = pd.to_datetime(X_financial['timestamp'])
X_financial = X_financial[['close', 'volume', 'ticker']]

X_financial.head()

In [21]:
# Choose start and end date for financial data so that is starts and ends in the spot for all available tickers
start_date = max([X_financial[X_financial['ticker'] == t].index[0] for t in X_financial['ticker'].unique()])
end_date   = max([X_financial[X_financial['ticker'] == t].index[-1] for t in X_financial['ticker'].unique()])

X_financial = X_financial.loc[X_financial.index >= start_date]
X_financial = X_financial.loc[X_financial.index <= end_date]

### Textual feature engineering

In [ ]:
# Available tickers to invest (from the financial .csv) one are
TICKERS

In [ ]:
def convert_date(date_string):
    date_obj = datetime.strptime(date_string, "%a, %d %b %Y %H:%M:%S %z")
    # Replace with the desired date and time
    return date_obj.strftime("%Y-%m-%d %H:%M:%S")

# Load the data
X_text = pd.concat(
    [load_data(j, type = 'Text/')[['text', 'date', 'sentiment', 'tickers']] for j in range(14)],
    axis = 0
)

# Choose only rows with at least one ticker mentioning some ticker in TICKERS
filtered_rows = X_text[X_text['tickers'].apply(lambda x: not set(eval(x)).isdisjoint(TICKERS))].copy()
filtered_rows['tickers'] = filtered_rows['tickers'].apply(lambda x: [ticker for ticker in eval(x) if ticker in TICKERS])

# Choose only important columns
filtered_rows = filtered_rows[['text', 'date', 'sentiment', 'tickers']]

# Format the date
filtered_rows['date'] = filtered_rows['date'].apply(convert_date)

# Bool columns for TICKERS
for ticker in TICKERS:
    filtered_rows[f'bool_{ticker}'] = filtered_rows['tickers'].apply(lambda x : ticker in x)

# Round *up* the date to a full minute
filtered_rows['date'] = pd.to_datetime(filtered_rows['date'])
filtered_rows['date'] = filtered_rows['date'].dt.ceil('min')

# Compute VADER sentiment scores
analyzer = SentimentIntensityAnalyzer()
filtered_rows['score'] = filtered_rows['text'].apply(lambda x: analyzer.polarity_scores(x)['compound'])

# Sort by date
filtered_rows = filtered_rows.sort_values(by = 'date', ascending = True)

In [24]:
melted = filtered_rows.melt(
    id_vars=['date', 'score', 'sentiment'], 
    value_vars=[f'bool_{ticker}' for ticker in TICKERS], 
    var_name='ticker', 
    value_name='is_present'
)

melted = melted[melted['is_present']]

melted['ticker'] = melted['ticker'].str.replace('bool_', '')

combined = melted.pivot_table(
    index='date',
    columns='ticker',
    values=['score', 'sentiment'],
    aggfunc='first'
)

combined.columns = [f"{col[1]}_{col[0]}" for col in combined.columns]

combined = combined.reset_index().sort_values(by='date')
combined.ffill(inplace=True)

for col in combined.columns:

    if '_sentiment' in col:
        combined[col] = combined[col].astype('string').fillna('Neutral')
    elif '_score' in col:
        combined[col] = combined[col].fillna(0)


# Ffill() wrt. the date column in minute frequency
combined['date'] = pd.to_datetime(combined['date'])
combined = combined.set_index(combined['date']).sort_index()
combined = combined.resample('1min').ffill()

# Outcome of hard work
X_nlp = combined.copy()

In [ ]:
X_financial

### Match the dates on `X_financial` and `X_nlp`

In [ ]:
temp_financial = X_financial[X_financial['ticker'] == TICKERS[0]].copy()
temp_financial

In [ ]:
all_data = []

for t in TICKERS:
    temp_financial = X_financial[X_financial['ticker'] == t].copy()
    temp_nlp = X_nlp[[f'{t}_score', f'{t}_sentiment']].copy()

    # Choose correct dates
    start_date = max(temp_financial.index[0], temp_nlp.index[0])
    end_date = min(temp_financial.index[-1], temp_nlp.index[-1])

    # Set dates in both dfs
    temp_financial = temp_financial.loc[temp_financial.index >= start_date]
    temp_financial = temp_financial.loc[temp_financial.index <= end_date]

    temp_nlp = temp_nlp.loc[temp_nlp.index >= start_date]
    temp_nlp = temp_nlp.loc[temp_nlp.index <= end_date]

    temp_financial = temp_financial.resample('1min').ffill()
    temp_nlp = temp_nlp.resample('1min').ffill()


    temp = pd.concat([temp_financial, temp_nlp], axis = 1)
    all_data.append(temp)

In [ ]:
# Good start for all
start_date = max([temp.index[0] for temp in all_data])
end_date = min([temp.index[-1] for temp in all_data])

all_data = [
    temp.loc[(temp.index >=start_date) & (temp.index <= end_date)]
    for temp in all_data
]

iter = 0
for temp in all_data:
    start_date = temp.index[0]
    end_date = temp.index[-1]
    temp.to_csv(f'Data/ParsedDataForModel/{TICKERS[iter]}.csv')
    print(f'Managed to save {TICKERS[iter]}.csv. Start date {start_date}, End date {end_date}')
    iter = iter + 1